# Prepare BATADAL dataset for GDN

GDN is an unsupervised anomaly detection algorithm that identifies anomalies at a timestep level for an entire system. This system consists of nodes that generate time-series (such as sensors in a water treatment plant), and GDN learns the relationship between the nodes during non-anomalous operation. These learned relationships can then be used at inference time to identify if the system is operating with anomalies.

To use GDN, we need a text file containing the list of nodes (will be referred to as 'sensors'), a train CSV containing timesteps and time series with for each sensor, and a test CSV containing timesteps and time series for each sensor, and labels for each time step.

This notebook prepares the text file, train and test CSVs from the original data.

In [1]:
import pandas as pd
import pathlib

In [2]:
source_dir = "../../data/01_raw/iot"
destination_dir = "../../data/03_primary/iot"

pathlib.Path(destination_dir).mkdir(parents=True, exist_ok=True)

In [3]:
train_no_anom = pd.read_csv(f"{source_dir}/BATADAL_dataset03_train_no_anomaly.csv")
train_some_anom = pd.read_csv(f"{source_dir}/BATADAL_dataset04_train_some_anomaly.csv")
test_with_anom = pd.read_csv(f"{source_dir}/BATADAL_test_dataset_some_anomaly.csv")

In [4]:
# has leading white space
train_some_anom.columns = train_some_anom.columns.str.strip()

In [5]:
train_no_anom.shape, train_some_anom.shape, test_with_anom.shape

((8761, 45), (4177, 45), (2089, 44))

# Train no anomaly

In [6]:
from sklearn.preprocessing import MinMaxScaler

In [7]:
SENSOR_COLS = [c for c in train_no_anom.columns if c not in ["DATETIME", "ATT_FLAG"]]

In [8]:
normalizer = MinMaxScaler(feature_range=(0, 1)).fit(train_no_anom[SENSOR_COLS])

In [9]:
train_no_anom[SENSOR_COLS] = normalizer.transform(train_no_anom[SENSOR_COLS])

In [10]:
with open(f"{destination_dir}/iot_sensor_list_batadal.txt", "w") as f:
    f.writelines("\n".join(SENSOR_COLS))

In [11]:
train_no_anom.reset_index().rename(columns={"index": "timestamp"})[["timestamp" ]+ SENSOR_COLS].to_csv(
    f"{destination_dir}/iot_gdn_train.csv",
    index=False
)

# Train with anomaly

In [12]:
from typing import Tuple, List
def append_anomaly_column(anomalies: List[Tuple], df: pd.DataFrame) -> pd.DataFrame:    
    fmt ="%d/%m/%Y %H"
    anomalies_dt = [
        (pd.to_datetime(s, format=fmt), pd.to_datetime(e, format=fmt)) for s, e in anomalies
    ]
    
    df = df.reset_index().rename(columns={"index": "timestamp"})
    df["pdDateTime"] = pd.to_datetime(df["DATETIME"], format="%d/%m/%y %H")
    df = df.set_index(["pdDateTime"])

    df["attack"] = 0
    for start, end in anomalies_dt:
        df.loc[start:end, "attack"] = 1
        
    return df

In [13]:
# from http://www.batadal.net/images/Attacks_TrainingDataset2.png
fmt ="%d/%m/%Y %H"
train_anomalies = [
    ("13/09/2016 23", "16/09/2016 00"),
    ("26/09/2016 11", "27/09/2016 10"),
    ("09/10/2016 09", "11/10/2016 20"),
    ("29/10/2016 19", "02/11/2016 16"),
    ("26/11/2016 17", "29/11/2016 04"),
    ("06/12/2016 07", "10/12/2016 04"),
    ("14/12/2016 15", "19/12/2016 04")
]

train_anomalies_dt = [
    (pd.to_datetime(s, format=fmt), pd.to_datetime(e, format=fmt)) for s, e in train_anomalies
]

In [14]:
train_some_anom = append_anomaly_column(train_anomalies_dt, train_some_anom)

In [15]:
train_some_anom["attack"].value_counts()

0    3685
1     492
Name: attack, dtype: int64

In [16]:
train_some_anom[SENSOR_COLS] = normalizer.transform(train_some_anom[SENSOR_COLS])

In [17]:
train_some_anom[["timestamp"] + SENSOR_COLS + ["attack"]].to_csv(
    f"{destination_dir}/iot_gdn_train_anom.csv", 
    index=False
)

# Test

In [18]:
# http://www.batadal.net/images/Attacks_TestDataset.png
test_anomalies = [
    ("16/01/2017 09", "19/01/2017 06"),
    ("30/01/2017 08", "02/02/2017 00"),
    ("09/02/2017 03", "10/02/2017 09"),
    ("12/02/2017 01", "13/02/2017 07"),
    ("24/02/2017 05", "28/02/2017 08"),
    ("10/03/2017 14", "13/03/2017 21"),
    ("25/03/2017 20", "27/03/2017 01")
]

test_anomalies_dt = [
    (pd.to_datetime(s, format=fmt), pd.to_datetime(e, format=fmt)) for s, e in test_anomalies
]

In [19]:
test_with_anom = append_anomaly_column(test_anomalies_dt, test_with_anom)

In [20]:
test_with_anom["attack"].value_counts()

0    1682
1     407
Name: attack, dtype: int64

In [21]:
test_with_anom[SENSOR_COLS] = normalizer.transform(test_with_anom[SENSOR_COLS])

In [22]:
test_with_anom[["timestamp"] + SENSOR_COLS + ["attack"]].to_csv(
    f"{destination_dir}/iot_gdn_test.csv", 
    index=False
)

# References
Riccardo Taormina and Stefano Galelli and Nils Ole Tippenhauer and Elad Salomons and Avi Ostfeld and Demetrios G. Eliades and Mohsen Aghashahi and Raanju Sundararajan and Mohsen Pourahmadi and M. Katherine Banks and B. M. Brentan and Enrique Campbell and G. Lima and D. Manzi and D. Ayala-Cabrera and M. Herrera and I. Montalvo and J. Izquierdo and E. Luvizotto and Sarin E. Chandy and Amin Rasekh and Zachary A. Barker and Bruce Campbell and M. Ehsan Shafiee and Marcio Giacomoni and Nikolaos Gatsis and Ahmad Taha and Ahmed A. Abokifa and Kelsey Haddad and Cynthia S. Lo and Pratim Biswas and M. Fayzul K. Pasha and Bijay Kc and Saravanakumar Lakshmanan Somasundaram and Mashor Housh and Ziv Ohar; "The Battle Of The Attack Detection Algorithms: Disclosing Cyber Attacks On Water Distribution Networks." Journal of Water Resources Planning and Management, 144 (8), August 2018

Ailin Deng and Bryan Hooi. 2021. Graph Neural Network-Based Anomaly Detection in Multivariate Time Series. CoRR abs/2106.06947, (2021). Retrieved from https://arxiv.org/abs/2106.06947 